# 微分可能LUTモデルによるMNIST学習

Stochasticモデルに BatchNormalization や Binarize(backward時はHard-Tanh)を加えることで、より一般的なデータに対してLUT回路学習を行います。 

## 事前準備

In [1]:
import numpy as np
from tqdm.notebook import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb

異なる閾値で2値化した画像でフレーム数を水増ししながら学習させます。この水増しをバイナリ変調と呼んでいます。

ここではフレーム方向の水増し量を frame_modulation_size で指定しています。

In [2]:
# configuration
epochs                = 4
net_name              = 'MnistAeDifferentiableLutCnn'
data_path             = './data/' + net_name
frame_modulation_size = 15

データセットは PyTorch の torchvision を使います。ミニバッチのサイズは64です。
BinaryBrainではミニバッチをフレーム数として FrameBufferオブジェクトで扱います。
バイナリ変調で計算中にフレーム数が変わるためデータセットの準備観点でのミニバッチと呼び分けています。

In [3]:
# dataset
dataset_train = torchvision.datasets.MNIST(root='./data/', train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root='./data/', train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=64, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=64, shuffle=False, num_workers=2)

## ネットワークの構築

DifferentiableLut に特に何もオプションをつけなければOKです。<br>
バイナリ変調を施すためにネットの前後に RealToBinary層とBinaryToReal層を入れています。<br>
send_command で "binary true" を送ることで、DifferentiableLut の内部の重み係数が 0.0-1.0 の間に拘束されます。

In [4]:
# define network
net = bb.Sequential([
            bb.RealToBinary(frame_modulation_size=frame_modulation_size),
            bb.DifferentiableLut([1024]),
            bb.DifferentiableLut([512]),
            bb.DifferentiableLut([10]),
            bb.BinaryToReal(frame_modulation_size=frame_modulation_size)
        ])
net.set_input_shape([1, 28, 28])

net.send_command("binary true")

loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam()

optimizer.set_variables(net.get_parameters(), net.get_gradients())

## 学習の実施

load_networks/save_networks で途中結果を保存/復帰可能できます。ネットワークの構造が変わると正常に読み込めなくなるので注意ください。
(その場合は新しいネットをsave_networksするまで一度load_networks をコメントアウトください)

tqdm などを使うと学習過程のプログレス表示ができて便利です。

In [5]:
# bb.load_networks(data_path, net)

# learning
for epoch in range(epochs):
    # learning
    loss.clear()
    metrics.clear()
    with tqdm(loader_train) as t:
        for images, labels in t:
            x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

            y_buf = net.forward(x_buf, train=True)
            
            dy_buf = loss.calculate(y_buf, t_buf)
            metrics.calculate(y_buf, t_buf)
            
            net.backward(dy_buf)

            optimizer.update()
            
            t.set_postfix(loss=loss.get(), acc=metrics.get())

    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)

    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))

    bb.save_networks(data_path, net, keep_olds=3)


epoch[0] : loss=1.719779 accuracy=0.749000



epoch[1] : loss=1.733394 accuracy=0.719900



epoch[2] : loss=1.738112 accuracy=0.722800



epoch[3] : loss=1.741070 accuracy=0.725600


## FPGA用Verilog出力

最後に学習したネットワークを Verilog 出力します。
MNISTのサイズである 28x28=784bit の入力を 10bit の分類をして出力するだけのシンプルなモジュールを出力します。

In [6]:
# export verilog
bb.export_verilog_lut_layers('./data/' + net_name + '.v', net_name, net)

## モデルの係数を取得する

Verilog以外の言語やFPGA以外に適用したい場合、接続とLUTテーブルの2つが取得できれば同じ計算をするモデルをインプリメントすることが可能です。

### 事前準備
そのままだと勾配はリセットされているので少しだけ逆伝搬を実施します

In [7]:
# 最新の保存データ読み込み
bb.load_networks(data_path, net)

# 1回だけ学習(勾配を作る)
for images, labels in loader_test:
    x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
    t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
    y_buf = net.forward(x_buf, train=True)
    net.backward(loss.calculate(y_buf, t_buf))
    break

# layer を取り出す
layer0 = net[1]
layer1 = net[2]
layer2 = net[3]

load : ./data/MnistDifferentiableLutSimple\20201229_181547


### 接続を取得する

LUTモデルは get_connection_list() にて接続行列を取得できます。<br>
ここでの各出力ノードは、6つの入力と接続されており、layer0 の出力ノードは 1024 個あるので、1024x6 の行列が取得できます。

In [8]:
connection_mat = np.array(layer0.get_connection_list())
print(connection_mat.shape)
connection_mat

(1024, 6)


array([[285, 227, 432, 459, 774, 502],
       [246,   4, 148, 633, 287, 638],
       [590, 679, 175, 507, 123, 196],
       ...,
       [652, 689, 385, 238, 444, 693],
       [214, 732, 514,  31,  30, 410],
       [548, 372, 435, 273,   8, 598]])

### FPGA化する場合のLUTテーブルを取得する

LUT化する場合のテーブルを取得します。<br>
6入力のLUTモデルなので $ 2^6 = 64 $ 個のテーブルがあります。<br>
モデル内に BatchNormalization 等を含む場合はそれらも加味して最終的にバイナリLUTにする場合に適した値を出力します。

In [9]:
lut_mat = np.array(layer0.get_lut_table_list())
print(lut_mat.shape)
lut_mat

(1024, 64)


array([[ True,  True,  True, ...,  True, False,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [False, False,  True, ...,  True,  True, False],
       ...,
       [False, False,  True, ..., False, False, False],
       [False,  True, False, ..., False, False,  True],
       [ True, False,  True, ..., False,  True, False]])

### 重み行列を覗いてみる

6入力のLUTモデルなので $ 2^6 = 64 $ 個のテーブルがあります。<br>
W() にて bb.Tensor 型で取得可能で、numpy() にて ndarray に変換できます。

In [10]:
W = layer0.W().numpy()
print(W.shape)
W

(1024, 64)


array([[0.50451314, 0.53728974, 0.4888476 , ..., 0.4949037 , 0.48859444,
        0.4984435 ],
       [0.49471307, 0.54348856, 0.49056014, ..., 0.5668912 , 0.5265722 ,
        0.55751145],
       [0.4995032 , 0.4875714 , 0.50896   , ..., 0.5085959 , 0.5111007 ,
        0.5108401 ],
       ...,
       [0.5089024 , 0.4808616 , 0.5062268 , ..., 0.48116052, 0.50471467,
        0.4851499 ],
       [0.5080077 , 0.50120825, 0.47873867, ..., 0.4891284 , 0.5002959 ,
        0.5140413 ],
       [0.49439397, 0.4887342 , 0.591726  , ..., 0.37645647, 0.5268289 ,
        0.4674572 ]], dtype=float32)

### 勾配を覗いてみる

同様に dW() でW の勾配が取得できます

In [11]:
dW = layer0.dW().numpy()
print(dW.shape)
dW

(1024, 64)


array([[ 1.3881879e-03,  4.6914932e-04,  5.5459817e-04, ...,
        -3.4577737e-05, -3.3939104e-05, -1.0666679e-05],
       [ 3.9503176e-04, -2.6271373e-06,  1.3167727e-04, ...,
        -1.9930480e-06,  7.7586274e-06, -6.6434882e-07],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 4.0079337e-03, -6.8234312e-03,  3.2267887e-03, ...,
        -8.6396409e-05,  3.0160099e-04, -1.4292425e-04]], dtype=float32)